In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [2]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
data=pd.read_csv('../hindi.csv')

In [4]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())


In [5]:
with open('pickles/df.pkl','rb') as file:
    DF=pickle.load(file)
    file.close()
    
with open('pickles/file_idx.pkl','rb') as file:
    file_idx=pickle.load(file)
    file.close()
    
with open('pickles/doc_words.pkl','rb') as file:
    doc_words=pickle.load(file)
    file.close()
    
with open('pickles/doc_norm.pkl','rb') as file:
    doc_norm=pickle.load(file)
    file.close()

In [6]:
def tfidf(query,counter=10):
    text = remove_special_characters(query)
    words = text.split()
    words=[word for word in words if word not in stopwords]
    words=[word for word in words if word in DF.keys()]

    q=[]
    q_norm=0
    for w in words:
        tf_idf=(words.count(w)*math.log(len(file_idx)/DF[w]))
        q.append(tf_idf)
        q_norm+=tf_idf**2
    q_norm=math.sqrt(q_norm)

    q=np.array(q)/q_norm

    score={}

    for i in range(len(file_idx)):
        doc_v=[]
        for w in words:
            tf_idf=(doc_words[i].count(w)*math.log(len(file_idx)/DF[w]))
            doc_v.append(tf_idf)
        doc_v=np.array(doc_v)/doc_norm[i]
        score[i]=np.dot(q,doc_v)

    score=sorted(score.items(),key=lambda x:x[1],reverse=True)
    verse_index=[]
    for i in score:
        if counter== 0:
            break
        verse_index.append(i[0])
        counter-=1
    return verse_index
    

In [8]:
verse_index=tfidf('भीष्म पितामह अपने भतीजे दुर्योधन')
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
9,1,12,तस्य संजनयन्हर्षं कुरुवृद्धः पितामहः।\n\nसिंहन...,भीष्म पितामह अपने भतीजे दुर्योधन के भीतर के भय...
7,1,10,अपर्याप्तं तदस्माकं बलं भीष्माभिरक्षितम्।\n\nप...,दुर्योधन के आत्म प्रशंसा करने वाले ये शब्द मिथ...
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,दुर्योधन यह समझ गया था कि भीष्म पितामह मानसिक ...
47,2,6,न चैतद्विद्मः कतरन्नो गरीयो\n\nयद्वा जयेम यदि ...,जब किसी अति उपयुक्त कार्य प्रणाली का आंकलन किय...
434,11,26,अमी च त्वां धृतराष्ट्रस्य पुत्राः\n\nसर्वे सहै...,इस श्लोक में अर्जुन इसका उल्लेख करता है कि भगव...
435,11,27,वक्त्राणि ते त्वरमाणा विशन्ति\n\nदंष्ट्राकराला...,इस श्लोक में अर्जुन इसका उल्लेख करता है कि भगव...
6,1,7,अस्माकं तु विशिष्टा ये तान्निबोध द्विजोत्तम।\n...,यहाँ कौरव सेना के प्रधान सेनापति द्रोणाचार्य क...
52,2,11,श्री भगवानुवाच\n\nअशोच्यानन्वशोचस्त्वं प्रज्ञा...,इस श्लोक के साथ श्रीकृष्ण अपने दिव्य उपदेश का ...
1,1,2,सञ्जय उवाच\n\nदृष्ट्वा तु पाण्डवानीकं व्यूढं द...,धृतराष्ट्र इस बात की पुष्टि करना चाहता था कि क...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
